In [ ]:
print('Setup complete.')


# Cross‑Provider Basics

This notebook is Colab‑ready and demonstrates:
- **OpenAI streaming** output in Python.
- Conditional handling for providers that **do not support streaming** (e.g., AskSage).

In [ ]:
# Install required packages for Colab
!pip install  requests 
# If you're on Colab, this will ensure the latest OpenAI SDK is available.
!pip -q install --upgrade openai



In [ ]:
# Read OPENAI_API_KEY from Colab Secrets (left sidebar → 🔑 Secrets)
# 1) Add a secret named OPENAI_API_KEY
# 2) Toggle "Notebook access" ON for this notebook
# 3) Then run this cell
import os
try:
    from google.colab import userdata  # Colab-only
    key = userdata.get("OPENAI_API_KEY")
    if key:
        os.environ["OPENAI_API_KEY"] = key
        print("✓ OPENAI_API_KEY is set from Colab Secrets")
    else:
        print("✗ OPENAI_API_KEY not found in Colab Secrets. Add it in the left sidebar (🔑).")
except Exception as e:
    print("⚠️ Could not import/use google.colab.userdata. Are you running outside Colab?")
    print("Error:", str(e))


In [ ]:

from dataclasses import dataclass
from typing import Optional, Literal

Provider = Literal["openai"]

@dataclass
class ChatInput:
    provider: Provider = "openai"
    model: str = "gpt-5-mini"
    system: Optional[str] = None
    user: str = "Say hello and count to 10 slowly."
    stream: bool = True  # request streaming where supported

SUPPORTED_STREAMING: dict[Provider, bool] = {
    "openai": True,
}

def provider_supports_streaming(provider: Provider) -> bool:
    return SUPPORTED_STREAMING.get(provider, False)


In [ ]:
from dataclasses import dataclass
from typing import Iterator, Optional
from openai import OpenAI
from contextlib import contextmanager
import sys

# Initialize client (ensure OPENAI_API_KEY is set, e.g., via Colab Secrets)
client = OpenAI()

@dataclass
class ChatInput:
    model: str = "gpt-5-mini"
    system: Optional[str] = None
    user: str = "Say hello and count to 10 slowly."

def print_streamed_text(text_iter: Iterator[str]) -> str:
    """Print tokens as they stream and return the final concatenated text."""
    final = []
    for chunk in text_iter:
        sys.stdout.write(chunk)
        sys.stdout.flush()
        final.append(chunk)
    print()  # newline after stream
    return "".join(final)

def stream_openai(chat: ChatInput) -> str:
    """
    True streaming using the Responses API event stream:
      - iterate events
      - handle response.output_text.delta
      - stop when response.completed
      - return the concatenated text (also accessible from get_final_response()).
    """
    # Prepare request payload
    request_input = (
        chat.user if chat.system is None else
        [
            {"role": "system", "content": chat.system},
            {"role": "user", "content": chat.user},
        ]
    )

    @contextmanager
    def response_stream():
        s = client.responses.stream(
            model=chat.model,
            input=request_input,
        )
        try:
            yield s
        finally:
            s.close()

    deltas: list[str] = []

    with response_stream() as s:
        for event in s:
            if event.type == "response.output_text.delta":
                deltas.append(event.delta)
                # print token as it arrives
                sys.stdout.write(event.delta)
                sys.stdout.flush()
            elif event.type == "response.completed":
                print()  # newline after final token

        # If you want usage/metadata, fetch the final response object:
        _final_response = s.get_final_response()
        return "".join(deltas)



## Demo — OpenAI (streaming)


In [ ]:

# --- Demo OpenAI ---
chat = ChatInput(
    model="gpt-5-mini",
    system="You are concise and helpful.",
    user="Explain in 2 short sentences what streaming is, then emit a tiny haiku.",
)

final_text = stream_openai(chat)
print("\n---\nFinal captured text:\n", final_text)



### Notes & References

- **OpenAI Responses API** basics and hosted tools examples: see the official Cookbook page.  
- **Streaming events** with the Responses API and new features are described in the OpenAI blog post (May 21, 2025).
